```
Regression Modeling in Practice Course
Wesleyan University

Logistic Regression Model
Mario Colosso V.

The sample comes from Cortez and Morais study about predicting forest fires using 
metereological data [Cortez and Morais, 2007]. The study includes data from 517
forest fires in the Natural Park Montesinho (Trás-os-Montes, in northeastern Portugal)
January 2000 to December 2003, including meteorological data, the type of vegetation
involved (which determines the six components of the Canadian Forest Fire Weather Index
(FWI) system --see below--) and the total burned area in order to generate a model capable
of predicting the burned area of small fires, which are more frequent.

Measures
The data contains:
* X, Y: location of the fire (x,y axis spatial coordinate within the Montesinho park map:
  from 1 to 9)
* month, day: month and day of the week the fire occurred (january to december and monday
  to sunday)
* FWI system components:
  - FFMC: Fine Fuel Moisture Code (numeric rating of the moisture content of litter and
    other cured fine fuels: 18.7 to 96.2)
  - DMC: Duff Moisture Code (numeric rating of the average moisture content of loosely
    compacted organic layers of moderate depth: 1.1 to 291.3)
  - DC: Drought Code (numeric rating of the average moisture content of deep, compact
    organic layers: 7.9 to 860.6)
  - ISI: Initial Spread Index (numeric rating of the expected rate of fire spread: 0.0
    to 56.1)
* Metereological variables:
  - temp: temperature (2.2 to 33.3 °C)
  - RH: relative humidity (15 to 100%)
  - wind: wind speed (0.4 to 9.4 Km/h)
  - rain: outside rain (0.0 to 6.4 mm/m^2)
* area: the burned area of the forest as response variable (0.0 to 1090.84 Ha).

```

# Forest Fires

In [1]:
# Import required libraries and set global options

import pandas
import numpy
import matplotlib.pyplot as plt
import seaborn
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import outliers_influence

pandas.set_option('display.float_format', lambda x:'%.3f'%x)

# Test categorical explanatory variables with more than two categories

In [2]:
# Load Forest Fires .csv file
fires = pandas.read_csv('forestfires.csv')

# DATA MANAGEMENT

# Delete rows where any or all of the data are missing
fires = fires.dropna()

# Convert categorical variables (months and days) into numerical values
months_table = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
                'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
days_table   = ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']

fires['month'] = [months_table.index(month) for month in fires['month'] ]
fires['day']   = [days_table.index(day)     for day   in fires['day']   ]

fires_attributes  = list(fires.columns.values)
number_of_columns = len(fires_attributes)

# Shift (X, Y) coordinates to origin
fires['X'] -= min(fires['X'])
fires['Y'] -= min(fires['Y'])


# TEST CATEGORICAL EXPLANATORY VARIABLE WITH MORE THAN TWO CATEGORIES

model = smf.ols(formula = "area ~ C(X) + C(Y) + C(month) + C(day) + FFMC + DMC + DC + ISI + temp + RH + wind + rain", 
                data = fires).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   area   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.8898
Date:                Tue, 28 Jun 2016   Prob (F-statistic):              0.663
Time:                        21:49:24   Log-Likelihood:                -2862.3
No. Observations:                 517   AIC:                             5805.
Df Residuals:                     477   BIC:                             5975.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         10.9942     72.125      0.

## Comments:

* Only DC and DMC features (Drought Code and Duff Moisture Code) are statistically relevant to predict burned area (p-values are 0.036 and 0.030 respectively)

* xxx

# Data Management

In [3]:
# Load Forest Fires .csv file
fires = pandas.read_csv('forestfires.csv')

In [4]:
# Delete rows where any or all of the data are missing
fires = fires.dropna()

In [5]:
# Convert categorical variables (months and days) into numerical values
fires = pandas.get_dummies(fires, prefix_sep = '_')

fires_attributes  = list(fires.columns.values)
number_of_columns = len(fires_attributes)

In [6]:
# Shift (X, Y) coordinates to origin
fires['X'] -= min(fires['X'])
fires['Y'] -= min(fires['Y'])

Logistic regression was developed by statistician David Cox in 1958. The binary logistic model is used to estimate the probability of a **binary response** based on one or more predictor (or independent) variables (features). (Reference: [Wikipedia](https://en.wikipedia.org/wiki/Logistic_regression))

In [7]:
# Convert target variable (burned area) into a categorical (binary) variable
# 0 = no burned area; 1 = some extension of the forest was burned
index_list = fires[fires['area'] > 0.].index.tolist()
fires.loc[index_list, 'area'] = 1.

In [8]:
# Center each explanatory variables
to_be_centered = fires_attributes[fires_attributes.index('FFMC') : 
                                  fires_attributes.index('rain') + 1]
for attr in to_be_centered:   #From FFMC to rain: Exclude categorical variables
    fires[attr] = fires[attr] - fires[attr].mean()

In [9]:
# Display general info about adjusted dataset
fires.describe().T

,count,mean,std,min,25%,50%,75%,max
X,517.000,3.669,2.314,0.000,2.000,3.000,6.000,8.000
Y,517.000,2.300,1.230,0.000,2.000,2.000,3.000,7.000
FFMC,517.000,0.000,5.520,-71.945,-0.445,0.955,2.255,5.555
DMC,517.000,-0.000,64.046,-109.772,-42.272,-2.572,31.528,180.428
DC,517.000,0.000,248.066,-540.040,-110.240,116.260,165.960,312.660
ISI,517.000,-0.000,4.559,-9.022,-2.522,-0.622,1.778,47.078
temp,517.000,0.000,5.807,-16.689,-3.389,0.411,3.911,14.411
RH,517.000,0.000,16.317,-29.288,-11.288,-2.288,8.712,55.712
wind,517.000,-0.000,1.792,-3.618,-1.318,-0.018,0.882,5.382
rain,517.000,0.000,0.296,-0.022,-0.022,-0.022,-0.022,6.378


# Logistic regression

In [10]:
# Test full model
response_variable     = 'area'
explanatory_variables = [attr for attr in fires_attributes if attr != response_variable]
explanatory_vars      = ' + '.join(explanatory_variables)
model_formula         = response_variable + ' ~ ' + explanatory_vars

lreg1 = smf.logit(formula = model_formula, 
                  data = fires).fit(maxiter = 150)
print('\nMODEL:', model_formula, '\n')
print(lreg1.summary())

# odds ratios
print ("\nOdds Ratios")
#print (numpy.exp(lreg1.params))

# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (numpy.exp(conf))

         Current function value: 0.655941
         Iterations: 150

MODEL: area ~ X + Y + FFMC + DMC + DC + ISI + temp + RH + wind + rain + month_apr + month_aug + month_dec + month_feb + month_jan + month_jul + month_jun + month_mar + month_may + month_nov + month_oct + month_sep + day_fri + day_mon + day_sat + day_sun + day_thu + day_tue + day_wed 

                           Logit Regression Results                           
Dep. Variable:                   area   No. Observations:                  517
Model:                          Logit   Df Residuals:                      489
Method:                           MLE   Df Model:                           27
Date:                Tue, 28 Jun 2016   Pseudo R-squ.:                 0.05232
Time:                        21:49:26   Log-Likelihood:                -339.12
converged:                      False   LL-Null:                       -357.85
                                        LLR p-value:                   0.08700
              

C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


## Remove highly collinear features

As stated in [PennState, STATS 501-Regression Methods](https://onlinecourses.science.psu.edu/stat501/node/348), one way to reduce data-based multicollinearity is to collect aditional data under different experimental or observational conditions, which is not the current case. We'll use `variance_inflation_factor()` to determinate highly collinear features and remove one or more violating predictors from the regression model.

> **variance_inflation_factor(exog, exog_idx)**

> The variance inflation factor (VIF) is a measure for the increase of the variance of the parameter estimates if an additional variable, given by `exog_idx` is added to the linear regression. It is a measure for multicollinearity of the design matrix, `exog`.

> One recommendation is that if VIF is greater than 5, then the explanatory variable given by `exog_idx` is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this.

> Reference: http://en.wikipedia.org/wiki/Variance_inflation_factor

In [11]:
fires_ck = numpy.array(fires)
fires_attr = list(params.index.values[1:])
highly_collinear_attr = list()
vif_list = list()
for attr in fires_attr:
    vif = outliers_influence.variance_inflation_factor(fires_ck, fires_attr.index(attr))
    vif_list.append(vif)
    if(vif > 5):
        highly_collinear_attr.append(attr)

print('\nVariance Inflation Factors:')
print(pandas.DataFrame(vif_list, index=fires_attr, columns=['VIF']).T)

print('\nHighly collinear features:')
print(highly_collinear_attr)


Variance Inflation Factors:
        X     Y  FFMC   DMC     DC   ISI  temp    RH  wind  rain   ...     \
VIF 1.523 1.521 2.288 3.911 26.836 1.816 4.551 2.804 1.288 1.092   ...      

     month_nov  month_oct  month_sep  day_fri  day_mon  day_sat  day_sun  \
VIF        inf        inf        inf      inf      inf      inf      inf   

     day_thu  day_tue  day_wed  
VIF      inf      inf      inf  

[1 rows x 29 columns]

Highly collinear features:
['DC', 'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'day_fri', 'day_mon', 'day_sat', 'day_sun', 'day_thu', 'day_tue', 'day_wed']


In [12]:
#
# Define some useful functions
#

def generate_logistic_model():
    global explanatory_variables
    
    response_variable     = 'area'
    explanatory_variables = [attr for attr in explanatory_variables if attr not in highly_collinear_attr]
    explanatory_vars      = ' + '.join(explanatory_variables)
    model_formula         = response_variable + ' ~ ' + explanatory_vars

    model = smf.logit(formula = model_formula, 
                      data = fires).fit(maxiter = 150)
    print()
    print('MODEL:', model_formula, '\n')
    print(model.summary())
    print()

    # odds ratios
    print ("Odds Ratios")
    #print (numpy.exp(lreg1.params))
    
    # odd ratios with 95% confidence intervals
    params = model.params
    conf = model.conf_int()
    conf['OR'] = params
    conf.columns = ['Lower CI', 'Upper CI', 'OR']
    print (numpy.exp(conf))


def reduce_collinear_features():
    global explanatory_variables
    global highly_collinear_attr
    
    fires_ck = numpy.array(fires)
    highly_collinear_attr = list()
    vif_list = list()
    for attr in explanatory_variables:
        vif = outliers_influence.variance_inflation_factor(fires_ck, explanatory_variables.index(attr))
        vif_list.append(vif)
        if(vif > 5):
            highly_collinear_attr.append(attr)

    print()
    print('Variance Inflation Factors:')
    print(pandas.DataFrame(vif_list, index=explanatory_variables, columns=['VIF']).T)

    print()
    print('Highly collinear features:')
    print(highly_collinear_attr)
    print()

    return(highly_collinear_attr != [])

In [13]:
# Generate a logistic model with highly collinear features removed

generate_logistic_model()

Optimization terminated successfully.
         Current function value: 0.681859
         Iterations 5

MODEL: area ~ X + Y + FFMC + DMC + ISI + temp + RH + wind + rain + month_apr 

                           Logit Regression Results                           
Dep. Variable:                   area   No. Observations:                  517
Model:                          Logit   Df Residuals:                      506
Method:                           MLE   Df Model:                           10
Date:                Tue, 28 Jun 2016   Pseudo R-squ.:                 0.01488
Time:                        21:49:27   Log-Likelihood:                -352.52
converged:                       True   LL-Null:                       -357.85
                                        LLR p-value:                    0.3856
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.2023      0

In [14]:
# Remove highly collinear features until there are no more and generate logistic regression model

while reduce_collinear_features():
    generate_logistic_model()
    print('\n--------------------------------------------------------------------------------')


Variance Inflation Factors:
        X     Y  FFMC   DMC    ISI  temp    RH  wind  rain  month_apr
VIF 1.523 1.521 2.288 3.911 26.836 1.816 4.551 2.804 1.288      1.092

Highly collinear features:
['ISI']

Optimization terminated successfully.
         Current function value: 0.682529
         Iterations 5

MODEL: area ~ X + Y + FFMC + DMC + temp + RH + wind + rain + month_apr 

                           Logit Regression Results                           
Dep. Variable:                   area   No. Observations:                  517
Model:                          Logit   Df Residuals:                      507
Method:                           MLE   Df Model:                            9
Date:                Tue, 28 Jun 2016   Pseudo R-squ.:                 0.01391
Time:                        21:49:27   Log-Likelihood:                -352.87
converged:                       True   LL-Null:                       -357.85
                                        LLR p-value:             

In [15]:
# Generate final logistic correlation model where all explanatory features independent
generate_logistic_model()

Optimization terminated successfully.
         Current function value: 0.686064
         Iterations 5

MODEL: area ~ X + Y + FFMC + DMC 

                           Logit Regression Results                           
Dep. Variable:                   area   No. Observations:                  517
Model:                          Logit   Df Residuals:                      512
Method:                           MLE   Df Model:                            4
Date:                Tue, 28 Jun 2016   Pseudo R-squ.:                0.008804
Time:                        21:49:28   Log-Likelihood:                -354.69
converged:                       True   LL-Null:                       -357.85
                                        LLR p-value:                    0.1778
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.1926      0.198     -0.973      0.331        -0.581    